## ch07. 앙상블 학습과 랜덤포레스트

앙상블 학습은 여러개의 분류기를 생성하여 그 예측을 결합함으로써 정확한 최종 예측을 도출하는 기법을 말한다.<br>
ex) 랜덤포레스트, 그래디언트 부스팅

<앙상블 학습의 유형>

1. 보팅(Voting) : 여러개의 다른 알고리즘을 동일한 데이터에 적용하여 최종예측
2. Bagging : Bootstrap Aggregation을 의미. 단일알고리즘을 여러개로 샘플링된 데이터에 적용하여 최종예측
3. Boosting : 가중치를 사용하여 약분류기를 강분류기로 만드는 과정을 의미한다. 분류기가 순차적으로 학습을 수행하며 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측할 수 있도록 다음 분류기에서는 가중치를 부여한다.
4. Stacking : 개별모델이 예측한 데이터를 다시 trainset이용, 새로운 메타모델에 적용한다

<br>
부스팅은 배깅보다 오류가 적지만 느리고, 오버피팅될 가능성이 높다. 무엇을 선택할 지는 상황에 따라 달라진다. 개별 결정 트리의 낮은 성능이 문제라면 부스팅이 적합하고, 오버 피팅이 문제라면 배깅이 적합하다.

### 7.1. 투표기반 분류기

**voting의 유형 - hardvoting과 softvoting** <br>
1. hardvoting : 예측한 결괏값들 중 다수의 분류기가 결정한 예측값을 최종 보팅의 결괏값으로 선정하는 것
2. softvoting : 분류기들의 레이블 값 결정확률을 모두 더하고 이를 평균하여 확률이 가장 높은 레이블 값을 최종 보팅 결괏값 채택
하드보팅보다는 소프트보팅의 예측성능이 좋아 더 많이 사용된다.

hard voting

In [1]:
#데이터 생성
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X,y=make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [2]:
#여러개의 분류기를 조합하여, voting분류기 만드는 경우
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression(solver='liblinear', random_state=42)
rnd_clf=RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf=SVC(gamma="auto", random_state=42)

#보팅 기반으로 여러개의 알고리즘 조합을 적용하는 경우
voting_clf=VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [3]:
#학습
voting_clf.fit(X,y)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,...
                                        

In [4]:
#정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.872
SVC 0.888
VotingClassifier 0.896


soft voting

In [5]:
#다른 데이터셋으로 소프트 보팅 적용하는 경우 
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
cancer=load_breast_cancer() #데이터셋 로드
data_df=pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


로지스틱 회귀와 knn을 기반으로 하여 소프트보팅 방식으로 새롭게 보팅분류기를 만들어본다 앙상블방법을 사용하여 분류하는데 사용되는 모델은 다양할 수 있다. 이에 대해 앙상블을 사용하되, 최종결정을 어떻게 내리느냐에 따라 나눠진다.  

In [9]:
import warnings
warnings.filterwarnings('ignore')
#로지스틱회귀와 KNN을 기반으로 하여 소프트 보팅방식으로 새롭게 보팅 분류기를 만들어 본다. 
#개별 모델은 KNN과 로지스틱이다.
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기 
vo_clf = VotingClassifier( estimators=[('LR',lr_clf),('KNN',knn_clf)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2 , random_state= 156)

# VotingClassifier 학습/예측/평가. 
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred))) #소프트부팅의 결과

Voting 분류기 정확도: 0.9474


In [10]:
#개별모델의 학습/예측/평가
classifiers=[lr_clf, knn_clf] #개별모델의 이름 저장 리스트

for classifier in classifiers : 
    classifier.fit(X_train, y_train)
    pred=classifier.predict(X_test)
    class_name=classifier.__class__.__name__
    print('{0} 정확도: {1: .4f}'.format(class_name, accuracy_score(y_test,pred))) #개별 결과

LogisticRegression 정확도:  0.9386
KNeighborsClassifier 정확도:  0.9386


### 7.2.배깅과 페이스팅
- **배깅**:훈련셋의 서브셋을 무작위로 구성하여 각기 다르게 분류기를 학습시키는 경우. 훈련셋에서 중복을 허용하여 샘플링하는 방식
- **같은 알고리즘으로 여러개의 분류기를 만들어 voting으로 최종 결정하는 알고리즘**
- **페이스팅**: 중복을 허용하지 않고 샘플링하여 모델을 적용하는 경우 <br>
- 예측모델을 돌린후에는 수집함수는 전형적으로 **분류**일 때, **통계적최빈값**을 따지며, **회귀**일때는 평균을 계산한다.<br>
- 또 앙상블의 결과, 편향과 분산이 모두 줄어든다. 

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9473684210526315


### 7.2.2. oob평가

In [13]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True, random_state=40)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9582417582417583

In [14]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.956140350877193